In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This is a expansion of EDA where i try to compare PCA and truncated svd to better optimize the dimensionality reduction and inturn get better results
Find the detailed EDA HERE - https://www.kaggle.com/ash1706/exploring-moa-eda-pca-xgboost-improved

this approach targets the use of autoencoders Truncated Svd and PCA to compare the performance(log loss based of a xg boost model)

<p style="font-size:22px"><b>!!!!!!!if you liked it please leave an Upvote 🙌🙌🙌🙌!!!!!!!

# Objective

> **MoA has too many columns and too sparse data and hence dimensionality reduction is the go to option. Mostly when it comes to dimensionality reduction PCA is our go to. But what about some other dimension reduction techniques as well. The objective of this kernel is to explore dimensionality reduction and compare them with PCA to get a sense of how they work and if we can get better results by using them.**

**Dimensionality reduction is an unsupervised learning technique.**

Nevertheless, it can be used as a data transform pre-processing step for machine learning algorithms on classification and regression predictive modeling datasets with supervised learning algorithms.

There are many dimensionality reduction algorithms to choose from and no single best algorithm for all cases. Instead, it is a good idea to explore a range of dimensionality reduction algorithms and different configurations for each algorithm.

We will cover the following techniques in this kernel:

1. PCA 
2. TRUNCATED-SVD 




In [ ]:
import pandas as pd 
import numpy as np

import seaborn as sns
sns.set_style("dark")
import matplotlib.pyplot as plt


from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

from time import time

import os

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:


from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss , accuracy_score

import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputClassifier

import os
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df_train = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
df_test = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
train_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train_non_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
SEED = 42
NFOLDS = 5
DATA_DIR = '/kaggle/input/lish-moa/'
np.random.seed(SEED)

data prep

In [ ]:
features = df_train

In [ ]:
common  = ['sig_id',
 'cp_type',
 'cp_time',
 'cp_dose']


genes = list(filter(lambda x : "g-" in x  , list(features)))

cells = list(filter(lambda x : "c-" in x  , list(features)))

In [ ]:
train = df_train
train['type'] = 'train'
test = df_test
test['type'] = 'test'
X = train.append(test)
X = pd.get_dummies(columns = ['cp_type' , 'cp_dose', 'cp_time'], drop_first =True , data = X)
target  = train_scored.drop(['sig_id'] , axis =1)


# lets label encode cp_type , cp_dose and cp_time
# X = pd.get_dummies(columns = ['cp_type' , 'cp_dose', 'cp_time'], drop_first =True , data = X)
# X[numeric_cols] = StandardScaler().fit_transform(X[numeric_cols])
numeric_cols = genes+cells


In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))
# classifier = MultiOutputClassifier(XGBClassifier())

clf = Pipeline([
                ('classify', classifier)
               ])


In [ ]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [ ]:
def modeling_xg_boost(x, y, x_test):
    oof_preds = np.zeros(y.shape)
    test_preds = np.zeros((test.shape[0], y.shape[1]))
    oof_losses = []
    kf = KFold(n_splits=NFOLDS)
    for fn, (trn_idx, val_idx) in enumerate(kf.split(x, y)):
        print('Starting fold: ', fn)
        X_train, X_val = x[trn_idx], x[val_idx]
        y_train, y_val = y[trn_idx], y[val_idx]
        
        # drop where cp_type==ctl_vehicle (baseline)
        ctl_mask = X_train[:,-4]==0
        X_train = X_train[~ctl_mask,:]
        y_train = y_train[~ctl_mask]
        
        clf.fit(X_train, y_train)
        val_preds = clf.predict_proba(X_val) # list of preds per class
        val_preds = np.array(val_preds)[:,:,1].T # take the positive class
        oof_preds[val_idx] = val_preds
        
        loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
        oof_losses.append(loss)
        preds = clf.predict_proba(x_test)
        preds = np.array(preds)[:,:,1].T # take the positive class
        test_preds += preds / NFOLDS
        
    print(oof_losses)
    print('Mean OOF loss across folds', np.mean(oof_losses))
    print('STD OOF loss across folds', np.std(oof_losses))
    control_mask = X[X['type'] =='train']['cp_type_trt_cp'] ==0
    oof_preds[control_mask] = 0

    print('\n OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))
    
    return  test_preds
    

# PCA

> PCA is a technique for reducing the number of dimensions in a dataset whilst retaining most information. It is using the correlation between some dimensions and tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed. PCA is mostly used for dense data which is not true in this case we also have a sparse PCA which scikit learn provodes.

PCA - GENES

In [ ]:
n_comp = [2,4,6,10 ,16,20,30,50,100,150,200,300,450,600] # list containing different values of components
explained = [] # explained variance ratio for each component of PCA
for x in n_comp:
    pca_gene = PCA(n_components=x)
    pca_gene.fit(X[genes])
    explained.append(pca_gene.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x,pca_gene.explained_variance_ratio_.sum()))
plt.plot(n_comp, explained)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()

At 450 we get satisfactory variance explaied of 92% this helps us reduce the dimensions from 770 to 450 without lossing much information.

**Cell PCA**

In [ ]:
n_comp = [1,2,4,6,8,10,16,20,30,50] # list containing different values of components
explained = [] # explained variance ratio for each component of PCA
for x in n_comp:
    pca_cell = PCA(n_components=x)
    pca_cell.fit(X[cells])
    explained.append(pca_cell.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x,pca_cell.explained_variance_ratio_.sum()))
plt.plot(n_comp, explained)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()

Far better than gene data PC2 has about 85% explained variance

Modelling 
1. gene pca cutoff - 450
2. cell pca cutoff -2

In [ ]:
#pca_gene data
pca_gene = PCA(n_components=450)
pca_gene_data = pca_gene.fit_transform(X[genes])
inter_pc_gene = pd.DataFrame(data = pca_gene_data)

Pca = X
transformed_genes = [str(i)+str('_gene') for i in list(inter_pc_gene) ]
Pca[transformed_genes] = inter_pc_gene[:]





In [ ]:
pca_cell = PCA(n_components=2)
pca_cell_data = pca_cell.fit_transform(X[cells])
inter_pc_cell = pd.DataFrame(data = pca_cell_data
             , columns = ['PC1', 'PC2'])
Pca['PC1_cell'] = inter_pc_cell['PC1']
Pca['PC2_cell'] = inter_pc_cell['PC2']

In [ ]:
features_final_pca = transformed_genes + ['PC1_cell', 'PC2_cell', 'cp_type_trt_cp', 'cp_dose_D2', 'cp_time_48', 'cp_time_72']

In [ ]:

x_pca = Pca[Pca['type']  == 'train'][features_final_pca].to_numpy()
y_pca = target.to_numpy()
x_test_pca = Pca[Pca['type']  == 'test'][features_final_pca].to_numpy()

In [ ]:
#  p = modeling_xg_boost(x_pca ,y_pca,x_test_pca )

***Here we get the log loss as 0.0175 lets compare this with other dimension reduction techniques***

# Truncated SVD 
PCA and T-SVD are both linear approaches the diffrence between them is that PCA works better for dense data and T-SVD is better for sparse data so high hopes for this one 🤞🤞

T-SVD genes

In [ ]:
n_comp = [2,4,6,10 ,16,20,30,50,100,150,200,300,450,600] # list containing different values of components
explained = [] # explained variance ratio for each component of PCA
for x in n_comp:
    svd_gene =TruncatedSVD(n_components=x)
    svd_gene.fit(X[genes])
    explained.append(svd_gene.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x,svd_gene.explained_variance_ratio_.sum()))
plt.plot(n_comp, explained)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()

Svd has a slightly beteer result of 93% for 450 componenets compared to PCA 92.9 % but still not any major diffrence to conclude anything

T-SVD cells

In [ ]:
n_comp = [1,2,4,6,8,10,16,20,30,50] # list containing different values of components
explained = [] # explained variance ratio for each component of PCA
for x in n_comp:
    svd_cell = TruncatedSVD(n_components=x)
    svd_cell.fit(X[cells])
    explained.append(svd_cell.explained_variance_ratio_.sum())
    print("Number of components = %r and explained variance = %r"%(x,svd_cell.explained_variance_ratio_.sum()))
plt.plot(n_comp, explained)
plt.xlabel('Number of components')
plt.ylabel("Explained Variance")
plt.title("Plot of Number of components v/s explained variance")
plt.show()

lets take 2 as cell cutoff here also for arguments sake

T-SVD modelling

In [ ]:
svd_gene = TruncatedSVD(n_components=450)
svd_gene_data = svd_gene.fit_transform(X[genes])
inter_pc_gene = pd.DataFrame(data = svd_gene_data)

Svd = X
transformed_genes = [str(i)+str('_gene') for i in list(inter_pc_gene) ]
Svd[transformed_genes] = inter_pc_gene[:]


In [ ]:
svd_cell = TruncatedSVD(n_components=2)
svd_cell_data =svd_cell.fit_transform(X[cells])
inter_pc_cell = pd.DataFrame(data = svd_cell_data
             , columns = ['PC1', 'PC2'])
Svd['PC1_cell'] = inter_pc_cell['PC1']
Svd['PC2_cell'] = inter_pc_cell['PC2']

In [ ]:
features_final_svd = transformed_genes + ['PC1_cell', 'PC2_cell', 'cp_type_trt_cp', 'cp_dose_D2', 'cp_time_48', 'cp_time_72']

In [ ]:
x_svd = Svd[Svd['type']  == 'train'][features_final_svd].to_numpy()
y_svd = target.to_numpy()
x_test_svd = Svd[Svd['type']  == 'test'][features_final_svd].to_numpy()

In [ ]:
# p2 = modeling_xg_boost(x_svd ,y_svd,x_test_svd )

both seem to be simmilar interms of log loss maybe if we can clean the data more it could give us better results.

lets look at deeplearning approches for dimension reduction

# Autoencoders for Dimensionality Reduction

Conventionally, autoencoders have been used for non-linear dimensionality reduction. Say we have a dataset where the number of features is way more than what we’d prefer it to be, we can use autoencoders to bring the feature set to the desired feature size through complex non-linear functions that we needn’t have to worry about! This is a more effective technique compared to linear dimensionality reduction methods like PCA (Principal Component Analysis) or other conventional non-linear techniques

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(X[numeric_cols])
data_scaled_genes = X[genes]
data_scaled_cells = X[cells]

In [ ]:
# Fixed dimensions
input_dim_genes = X[genes].shape[1]  # 8
encoding_dim_genes = 10
# Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
input_layer_genes = Input(shape=(input_dim_genes, ))
encoder_layer_1_genes = Dense(6, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer_genes)
encoder_layer_2_genes = Dense(4, activation="tanh")(encoder_layer_1_genes)
encoder_layer_3_genes = Dense(encoding_dim_genes, activation="tanh")(encoder_layer_2_genes)

In [ ]:
# Creat encoder model
encoder_genes = Model(inputs=input_layer_genes, outputs=encoder_layer_3_genes)
# Use the model to predict the factors which sum up the information of interest rates.
encoded_data_genes = pd.DataFrame(encoder_genes.predict(data_scaled_genes))
encoded_cols_list_gene = [str(i)+'_feature' for i in list(encoded_data_genes)]
encoded_data_genes.columns = encoded_cols_list_gene


In [ ]:
# Fixed dimensions
input_dim_cells = X[cells].shape[1]  # 8
encoding_dim_cells = 2
# Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
input_layer_cells = Input(shape=(input_dim_cells, ))
encoder_layer_1_cells = Dense(6, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer_cells)
encoder_layer_2_cells = Dense(4, activation="tanh")(encoder_layer_1_cells)
encoder_layer_3_cells = Dense(encoding_dim_cells, activation="tanh")(encoder_layer_2_cells)

In [ ]:
# Creat encoder model
encoder_cells = Model(inputs=input_layer_cells, outputs=encoder_layer_3_cells)
# Use the model to predict the factors which sum up the information of interest rates.
encoded_data_cells = pd.DataFrame(encoder_cells.predict(data_scaled_cells))
encoded_cols_list_cells = [str(i)+'_feature_cells' for i in list(encoded_data_cells)]
encoded_data_cells.columns = encoded_cols_list_cells


In [ ]:

final_encoded = encoded_data_cells
final_encoded[encoded_cols_list_gene] = encoded_data_genes[:]


In [ ]:
encoded_cols = encoded_cols_list_cells + encoded_cols_list_gene

In [ ]:
final_encoded[['cp_type_trt_cp', 'cp_dose_D2', 'cp_time_48', 'cp_time_72' , 'type']]   = X[['cp_type_trt_cp', 'cp_dose_D2', 'cp_time_48', 'cp_time_72' , 'type']].values

In [ ]:
feats = encoded_cols + ['cp_type_trt_cp', 'cp_dose_D2' , 'cp_time_48',  'cp_time_72']
x_encode = final_encoded[final_encoded['type']  == 'train'][feats].to_numpy()
y_encode = target.to_numpy()
x_test_encode = final_encoded[final_encoded['type']  == 'test'][feats].to_numpy()

In [ ]:
test_preds = modeling_xg_boost(x_encode ,y_encode,x_test_encode)

In [ ]:
# p

In [ ]:
# test_pred_f = (test_preds +p+p2)/3

In [ ]:
control_mask = final_encoded[final_encoded['type'] =='test']['cp_type_trt_cp'] == 0
test_preds[control_mask] = 0

In [ ]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
sub.iloc[:,1:] = test_preds
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()

Mainfold techniques like tsne are not suggestested because of high compute time and in this case it will be impracticle to use it 

# if you liked it please Upvote 😁  !!!!!!!
* I'll try to work more on data cleaning to highlight the diffrences more and maybe take a sample to explore mainfolding techniques like t-SNE 
* more work on autoencoders comming up !!!!!! 🙌


In [ ]:
##